## Tensorflow layers

In [ ]:
# default_exp layers

In [ ]:
# export
import tensorflow as tf

class MaskSequence(tf.keras.layers.Layer):
    """ Base class to prepare masked items inputs/ labels for language modeling tasks.
    
        Transformer architectures can be trained in different ways. Depending on the training method,
        there is a specifc msaking schema. The masking schema sets the items to be predicted (labels)
        and mask (hide) their positions in the sequence so that they are not used by the Transformer
        layers for prediction
    
        We currently provide 4 different masking schemes out of the box:
        - Causal LM (clm)
        - Masked LM (mlm)
        - Permutation LM (plm)
        - Replacement TOken (rtd)
        
        This class can be extended to add different a masking scheme.
        
        Parameters
        ----------
    """
    
    def __init__(self, padding_idx: int =0, eval_on_last_item_seq_only: bool=True, **kwargs):
        super(MaskSequence, self).__init__(**kwargs)
        self.padding_idx = padding_idx
        self.eval_on_last_item_seq_only = eval_on_last_item_seq_only
        self.mask_schema = None
        self.masked_targets = None
        
    def get_config(self):
        config = super(MaskSequence, self).get_config()
        config.update({
            'padding_idx': self.padding_idx,
            'eval_on_last_item_seq_only' : self.eval_on_last_item_seq_only
        })
    
    
        return config
    
    def _compute_masked_targets(self, item_ids: tf.Tensor, training=False):
        """ Method to prepare masked labels based on the sequence of item ids.
            It returns the True labels for masked positions and the related boolean mask.
            
            Parameters
            ----------
            item:ids: tf.Tensor
            The sequence fo input ids used for deriving labels of next item prediction task
            
            training:bool
            Flag to indicate whether we are in 'Training' or not.
        """
        raise NotImplementedError
        
    def compute_masked_targets(self, item_ids: tf.Tensor, training=False):
        """
        Method to prepare masked labels based on the sequence of item ids.
        It returns the true labels of masked positions and the related boolean mask.
        """
        
        assert item_ids.rank == 2, "item_ids must have 2 dimensions"
        masking_info == self._compute_masked_targets(item_ids, training=training)
        self.mask_schema, self.masked_targets = masking_info.schema, masking_info.targets
        
        
        return masking_info
    
    def apply_mask_to_inputs(self, inputs: tf.Tensor, schema:tf.Tensor)->tf.Tensor:
        """ 
        Control the masked predictions in the inputs by replacing the true interactions
        by a learnable masked embedding.
        
        Parameters
        ----------
        inputs: tf.Tensor
            The 3-D tensor of interaction embeddings resulting from the ops:
            
        schema: MaskingSchema
        """
        
        inputs = tf.where(
            tf.cast(tf.expand_dims(schema,-1), tf.bool),
            inputs,
            tf.cast(self.masked_item_embedding,dtype=inputs.dtype)
        )
        
        return inputs
    
    
    def call(self, inputs: tf.Tensor, item_ids: tf.Tensor, training=False)-> tf.Tensor:
        _ = self.compute_masked_targets(item_ids = item_ids, training=training)
        return self.apply_mask_to_inputs(inputs, self.mask_schema)
    
    def build(self, input_shape):
        self.hidden_size = input_shape[-1]
        
        # create a trainable embedding to replace masked interactions
        initializer = tf.random_normal_initializer(mean=0.0, stddev=0.001)
        self.masked_item_embedding = tf.Variable(
            initializer(shape=[self.hidden_size], dtype=tf.float32)
        )
        
        return super().build(input_shape)
    

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted 01_game.ipynb.
Converted 03_tensorflow.ipynb.
Converted 05_fizzbuzz.ipynb.
Converted 07_knowledgegraph.ipynb.
Converted 08_siamese.ipynb.
Converted index.ipynb.
Converted lib_layers.py.ipynb.
Converted lib_relation_extraction.ipynb.
Converted lib_span.ipynb.
Converted lib_wikiscrape.ipynb.
Converted multihead.ipynb.
